In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import GPT2Tokenizer, GPT2Model
import os
import re

# Download latest version
path = kagglehub.dataset_download("suchintikasarkar/sentiment-analysis-for-mental-health")

print("Path to dataset files:", path)

d:\ANACONDA\envs\hello_pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 11.1M/11.1M [00:01<00:00, 6.45MB/s]

Extracting files...


Path to dataset files: C:\Users\25769\.cache\kagglehub\datasets\suchintikasarkar\sentiment-analysis-for-mental-health\versions\1


In [ ]:


# Assuming the path printed from the kagglehub download is correct
# We need to find the specific CSV file within that downloaded directory.
# Based on the dataset name, the main data file is likely named 'mental_health_sentiment.csv'
# If the path structure is complex, you may need to adjust the file name/path.

# Replace 'path' with the actual path variable from your kagglehub download output
dataset_path = path # Example path, replace with actual output

file_name = "Combined Data.csv"
file_path = os.path.join(dataset_path, file_name)

print(f"Attempting to load data from: {file_path}")

try:
    # Load the dataset into a pandas DataFrame
    df = pd.read_csv(file_path)

    print("\n--- 1. DataFrame Head (First 5 Rows) ---")
    print(df.head())

    print("\n--- 2. DataFrame Information (Columns, Non-null counts, Dtypes) ---")
    print(df.info())

    print("\n--- 3. Dataset Shape (Rows, Columns) ---")
    print(df.shape)

    print("\n--- 4. Target Variable Distribution (Sentiment/Label) ---")
    # Assuming the target column is named 'label' or 'sentiment'
    # We will try 'label' based on similar datasets, you might need to adjust this.
    try:
        target_column = 'label'  # Adjust if the actual column name is different
        print(df[target_column].value_counts())
        print(f"\nTarget Variable Unique Values: {df[target_column].nunique()}")

    except KeyError:
        print("Could not find a 'label' column. Please check the actual column names from .head()")
        # Fallback: check all unique values in all object columns
        print("\nObject (Text) Column Unique Values Check:")
        for col in df.select_dtypes(include=['object']).columns:
             if df[col].nunique() < 20: # Likely a category column
                 print(f"Column '{col}' value counts:\n{df[col].value_counts()}\n")


    print("\n--- 5. Check for Missing Values ---")
    print(df.isnull().sum())


except FileNotFoundError:
    print(f"\nError: File not found at {file_path}. Please check the downloaded path and file name.")
except Exception as e:
    print(f"\nAn error occurred during data loading: {e}")

Attempting to load data from: C:\Users\25769\.cache\kagglehub\datasets\suchintikasarkar\sentiment-analysis-for-mental-health\versions\1\Combined Data.csv

--- 1. DataFrame Head (First 5 Rows) ---
   Unnamed: 0                                          statement   status
0           0                                         oh my gosh  Anxiety
1           1  trouble sleeping, confused mind, restless hear...  Anxiety
2           2  All wrong, back off dear, forward doubt. Stay ...  Anxiety
3           3  I've shifted my focus to something else but I'...  Anxiety
4           4  I'm restless and restless, it's been a month n...  Anxiety

--- 2. DataFrame Information (Columns, Non-null counts, Dtypes) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 

In [ ]:


# --- 1. Load Data (assuming you have the 'df' DataFrame from your output) ---

# Re-creating the DataFrame structure based on your output for demonstration
# In your actual environment, you'll use the existing 'df'
dataset_path = "C:\\Users\\25769\\.cache\\kagglehub\\datasets\\suchintikasarkar\\sentiment-analysis-for-mental-health\\versions\\1"
file_path = os.path.join(dataset_path, "Combined Data.csv")
df = pd.read_csv(file_path)

# Drop the irrelevant index column
df = df.drop(columns=['Unnamed: 0'])

# --- 2. Data Cleaning and Preprocessing ---

# Handle missing values by dropping rows where 'statement' is NaN
# Since only 362 out of 53043 are missing, dropping them is generally safe.
df.dropna(subset=['statement'], inplace=True)
print(f"Data shape after handling missing values: {df.shape}")

# Convert all text to string (ensures robustness)
df['statement'] = df['statement'].astype(str)

# --- 3. Label Encoding for the Target Variable (status) ---

# Initialize LabelEncoder
le = LabelEncoder()
# Fit and transform the 'status' column
df['status_encoded'] = le.fit_transform(df['status'])

# Display the mapping
print("\n--- Label Encoding Mapping ---")
for i, label in enumerate(le.classes_):
    print(f"{label}: {i}")

# --- 4. GPT-2 Feature Extraction Function ---

def get_gpt2_embeddings(texts, model, tokenizer, batch_size=32):
    """Generates sentence embeddings using GPT-2 by mean pooling."""

    # Set pad token for GPT-2 (it doesn't have one by default)
    # Using the EOS token as PAD token is a common workaround for feature extraction
    tokenizer.pad_token = tokenizer.eos_token

    # Place model in evaluation mode and move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_embeddings = []
    
    # Process texts in batches to manage memory
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize the batch
        # Padding='longest' for dynamic padding, Truncation=True to prevent sequence overflow
        # return_tensors='pt' returns PyTorch tensors
        encoded_input = tokenizer(
            batch_texts.tolist(), 
            padding='longest', 
            truncation=True, 
            max_length=128, # Choose a max length based on data, 128 is common
            return_tensors='pt'
        )
        
        # Move tensors to the appropriate device
        input_ids = encoded_input['input_ids'].to(device)
        attention_mask = encoded_input['attention_mask'].to(device)
        
        with torch.no_grad():
            # Get the model output
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            # The last hidden state contains the token embeddings
            embeddings = output.last_hidden_state # Shape: (Batch Size, Sequence Length, Hidden Dimension)

        # --- Mean Pooling to get Sentence Vector ---
        # Multiply embeddings by the attention mask to zero out padding tokens
        mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        masked_embeddings = embeddings * mask_expanded
        
        # Sum all non-padding tokens
        summed_embeddings = torch.sum(masked_embeddings, 1)
        
        # Calculate the actual number of tokens (non-padding)
        summed_mask = torch.clamp(attention_mask.sum(1), min=1e-9) # Avoid division by zero
        
        # Divide sum by count to get the mean (mean pooling)
        mean_pooled_embeddings = (summed_embeddings / summed_mask.unsqueeze(-1)).cpu().numpy()
        
        all_embeddings.append(mean_pooled_embeddings)

    return np.concatenate(all_embeddings, axis=0)


# Initialize GPT-2 components
print("\n--- Loading GPT-2 Model and Tokenizer (This may take a moment) ---")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Generate the feature vectors (X)
X_features = get_gpt2_embeddings(df['statement'], model, tokenizer, batch_size=64)
Y_labels = df['status_encoded'].values

print(f"\nGenerated Feature Matrix X shape: {X_features.shape}")
print(f"Generated Label Vector Y shape: {Y_labels.shape}")

# --- 5. Data Splitting and Logistic Regression ---

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X_features, Y_labels, test_size=0.2, random_state=42, stratify=Y_labels
)

print(f"\nTraining on {X_train.shape[0]} samples, Testing on {X_test.shape[0]} samples.")

# Initialize and train Logistic Regression model
print("\n--- Training Logistic Regression Model ---")
# Use a high max_iter and appropriate solver for large datasets/features
lr_model = LogisticRegression(max_iter=500, solver='sag', multi_class='multinomial', random_state=42)
lr_model.fit(X_train, Y_train)

# --- 6. Model Evaluation ---

# Predict on the test set
Y_pred = lr_model.predict(X_test)

print("\n--- Evaluation Results (Logistic Regression with GPT-2 Features) ---")
print(classification_report(Y_test, Y_pred, target_names=le.classes_))

Data shape after handling missing values: (52681, 2)

--- Label Encoding Mapping ---
Anxiety: 0
Bipolar: 1
Depression: 2
Normal: 3
Personality disorder: 4
Stress: 5
Suicidal: 6

--- Loading GPT-2 Model and Tokenizer (This may take a moment) ---


d:\ANACONDA\envs\hello_pytorch\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\25769\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better per


Generated Feature Matrix X shape: (52681, 768)
Generated Label Vector Y shape: (52681,)

Training on 42144 samples, Testing on 10537 samples.

--- Training Logistic Regression Model ---


d:\ANACONDA\envs\hello_pytorch\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



--- Evaluation Results (Logistic Regression with GPT-2 Features) ---
                      precision    recall  f1-score   support

             Anxiety       0.75      0.71      0.73       768
             Bipolar       0.76      0.71      0.73       556
          Depression       0.68      0.73      0.70      3081
              Normal       0.88      0.94      0.91      3269
Personality disorder       0.73      0.50      0.60       215
              Stress       0.55      0.43      0.48       517
            Suicidal       0.67      0.61      0.64      2131

            accuracy                           0.75     10537
           macro avg       0.72      0.66      0.68     10537
        weighted avg       0.74      0.75      0.74     10537



d:\ANACONDA\envs\hello_pytorch\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
